# Windows Process

In this section, we briefly cover the expected processes during boot and startup of a Windows system. Why is this important? Simply so we know what is normal, and what is (possibly) suspicious. As with all investigations, the analyst must have the understanding and context to say what is worth investigating and reporting on. Hopefully this gives some guidance to you. If you want the TLDR; I've included the simplified flow diagram below. If you want to modify it for your purpose, the Draw.IO file is also available in the root directory / drawings.<br>

![WindowsProcesses.png](../../Images/Goals/DFIR/WindowsProcesses.png)<br>

## Processes

Processes are simply programs running in the background of the system. Well, kinda, but it's easier to think of it this way for now. Processes can be seen in Task Manager on modern Windows versions, but a far more common tool is "Process Explorer", which forms a part of SysInternals. I don't dive into this here.

### System
The system process is the first launched after boot. It's role is to call all other processes used for booting the OS. As such it shouldn't have any parent processes and it is always process ID 4. You may be asking why the first one is #4, well I was wondering that too. Simply it's because process IDs are divisible by 4 due to a property of the process handles, the lower 2 bits of the ID are used for tags. Therefore the first PID after 0 is 4. Bit of trivia for you. Anyways....<br>
<br>

Expected Properties:<br>

> Process Count: One instance only<br>
> Parent Processes: No Parent process (there may be an exception for the idle process, not sure why) <br>
> User: Local System <br>
> Start Time: At Boot <br>
> Path: C:\Windows\System32\ntoskrnl.exe <br>
> Special Information: ALWAYS PID 4<br>

### SMSS
The Session Manager Sub System (SMSS) is also known as the Windows Session Manger. Don't confuse it with SSMS (Sql Server Management Studio) - I know I keep mixing them up... It is launched by the 'system' process and has a seperate instance for each 'session' on the machine. Session 0 is always reserved for the system. If you aren't sure what a 'session' is, I always relate it back to RDP. On a server, you can have multiple 'active' users logged into a system at once. Under the hood, the RDP session just launches a new session to be used. You can also use this to kill connections (worst case scenario) but you'll learn how important Session 0 is if you do this accidentally....<br>

Expected Properties:<br>

> Process Count: Multiple Possible <br>
> Parent Processes: System (PID4) <br>
> User: Local System <br>
> Start Time: At Boot for Session 0, login time for others <br>
> Path: C:\Windows\System32\smss.exe <br>
> Special Information: Launches SRSS and WinInit then terminates - shouldn't be running normally <br>

### CSRSS
The Client Server Runtime proceSS (yes, it uses the end of process). It is launched by SMSS and inherits its session number. As SMSS terminates after launching it's child processes, CSRSS shouldn't have a parent process known. Again, Session 0 is reserved for the OS and should be running.<br>

Expected Properties:<br>

> Process Count: Multiple Possible <br>
> Parent Processes: SMSS (should show no parent or parent not found) <br>
> User: Local System <br>
> Start Time: At Boot for Session 0, login time for others <br>
> Path: C:\Windows\System32\csrss.exe <br>
> Special Information: Inherets session number from SMSS<br>

### WinInit
The Windows Initalisation Process is another one launched by SMSS so the properties are expected to be about the same.<br>
Expected Properties:<br>

> Process Count: Just one <br>
> Parent Processes: SMSS (should show no parent or parent not found) <br>
> User: Local System <br>
> Start Time: At Boot <br>
> Path: C:\Windows\System32\wininit.exe <br>
> Special Information: Should match the launch time of CSRSS for session 0<br>

### SCM - Service Control Manager
The Services control manager run OS level services on the machine. These services are registered against the registry and stored in HKLM\System\CurrentControlSet\Services. <br>
Expected Properties:<br>

> Process Count: Single Only <br>
> Parent Processes: wininit.exe <br>
> User: Local System <br>
> Start Time: Shortly after boot <br>
> Path: C:\Windows\System32\services.exe<br>
> Special Information: Services can be queried with sc.exe (or via powershell, but sc has more fun stuff)<br>

### svchost.exe
This is the host process for windows services. When a service is launched (by SCM), it starts it's own process. This is a common place for malicious activity so keep an eye out here. Look for misspelling. Services should always be .dlls linked in HKLM\system\CurrentControlSet\services\\[service name]\parameters<br>
Expected Properties:<br>

> Process Count: Multiple, but only one per service. <br>
> Parent Processes: services.exe <br>
> User: User defined for the service <br>
> Start Time: At service start <br>
> Path: C:\Windows\System32\svchost.exe <br>
> Special Information: The correct way to call svchost is "svchost.exe -k [service name]"; the -k is a Key requirement (see what I did there?)<br>

### lsass.exe
The Local Security Authority SubSystem service enforces the security policy on the local machine. The Authentication process is in HKLM\system\CurrentControlSet\Control\lsa.<br>
Expected Properties:<br>

> Process Count: Single Instance <br>
> Parent Processes: wininit <br>
> User: system <br>
> Start Time: At Boot<br>
> Path: C:\Windows\System32\lsass.exe <br>
> Special Information: Should match the launch time of CSRSS for the same session<br>

### winlogin.exe
Winlogin.exe loads the SAS (Secure attention sequence) -> This is the Control + Alt + Delete prompt. Once login is successful, it loads the user profile. This includes loading the user's NTUSER.dat to HKCU. I then loads userinit then the user shell (explorer.exe).<br>
Expected Properties:<br>

> Process Count: Single Instance <br>
> Parent Processes: SMSS (should show no parent or parent not found) <br>
> User: Local System <br>
> Start Time: At Boot<br>
> Path: C:\Windows\System32\winlogin.exe <br>
> Special Information: the exe to run is in the registry at HKLM\software\Microsoft\WindowsNT\CurrentVersion\winlogon<br>

### explorer.exe
This is the start menu, taskbar etc etc. Explorer is also the GUI file browser.
> Process Count: Single Instance <br>
> Parent Processes: userinit (should show no parent or parent not found) <br>
> User: Logged in user <br>
> Start Time: At Login time<br>
> Path: C:\Windows\Explorer.exe <br>
> Special Information: Should have no TCP connections<br>

<hr>